In [1]:
pip install linearmodels

In [0]:
import pandas as pd
from linearmodels import IVGMM

In [3]:
file="https://github.com/VitorKamada/ECO7110/raw/master/Data/nls80.dta"
df = pd.read_stata(file)
df.head()

,wage,hours,iq,kww,educ,exper,tenure,age,married,black,south,urban,sibs,brthord,meduc,feduc,lwage
0,769,40,93,35,12,11,2,31,1,0,0,1,1,2.0,8.0,8.0,6.645091
1,808,50,119,41,18,11,16,37,1,0,0,1,1,NaN,14.0,14.0,6.694562
2,825,40,108,46,14,11,9,33,1,0,0,1,1,2.0,14.0,14.0,6.715384
3,650,40,96,32,12,13,7,32,1,0,0,1,4,3.0,12.0,12.0,6.476973
4,562,40,74,27,11,14,5,34,1,0,0,1,10,6.0,6.0,11.0,6.331502


In [4]:
Xs = ['exper', 'tenure', 'married', 'south','urban','black']
print(df[['lwage','educ','iq','kww'] + 
           Xs].describe(percentiles=[]))

            lwage        educ          iq  ...       south       urban       black
count  935.000000  935.000000  935.000000  ...  935.000000  935.000000  935.000000
mean     6.779002   13.468449  101.282353  ...    0.341176    0.717647    0.128342
std      0.421144    2.196654   15.052636  ...    0.474358    0.450385    0.334650
min      4.744932    9.000000   50.000000  ...    0.000000    0.000000    0.000000
50%      6.807935   12.000000  102.000000  ...    0.000000    1.000000    0.000000
max      8.032035   18.000000  145.000000  ...    1.000000    1.000000    1.000000

[6 rows x 10 columns]


In [0]:
df['const'] = 1
Xs = ['const'] + Xs

In [6]:
OLS = IVGMM(df.lwage, df[Xs+['educ']],
            None, None).fit()
print(OLS)

                            OLS Estimation Summary                            
Dep. Variable:                  lwage   R-squared:                      0.2526
Estimator:                        OLS   Adj. R-squared:                 0.2469
No. Observations:                 935   F-statistic:                    358.89
Date:                Tue, Aug 06 2019   P-value (F-stat)                0.0000
Time:                        14:32:22   Distribution:                  chi2(7)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          5.3955     0.1126     47.899     0.0000      5.1747      5.6163
exper          0.0140     0.0032     4.3548     0.00

In [7]:
Proxy = IVGMM(df.lwage, df[Xs+['educ']+['iq']],
              None, None).fit()
print(Proxy)

                            OLS Estimation Summary                            
Dep. Variable:                  lwage   R-squared:                      0.2628
Estimator:                        OLS   Adj. R-squared:                 0.2564
No. Observations:                 935   F-statistic:                    391.81
Date:                Tue, Aug 06 2019   P-value (F-stat)                0.0000
Time:                        14:32:22   Distribution:                  chi2(8)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          5.1764     0.1206     42.909     0.0000      4.9400      5.4129
exper          0.0141     0.0032     4.3896     0.00

In [8]:
IV_IQ = IVGMM(df.lwage, df[Xs],
     df.educ, df[['iq']]).fit()
print(IV_IQ)

                          IV-GMM Estimation Summary                           
Dep. Variable:                  lwage   R-squared:                      0.2106
Estimator:                     IV-GMM   Adj. R-squared:                 0.2047
No. Observations:                 935   F-statistic:                    285.63
Date:                Tue, Aug 06 2019   P-value (F-stat)                0.0000
Time:                        14:32:22   Distribution:                  chi2(7)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          4.6730     0.2194     21.300     0.0000      4.2430      5.1030
exper          0.0244     0.0043     5.7388     0.00

In [9]:
IV_IQ_KWW = IVGMM(df.lwage, df[Xs],
   df.educ, df[['iq','kww']]).fit()
print(IV_IQ_KWW)

                          IV-GMM Estimation Summary                           
Dep. Variable:                  lwage   R-squared:                      0.2152
Estimator:                     IV-GMM   Adj. R-squared:                 0.2092
No. Observations:                 935   F-statistic:                    310.47
Date:                Tue, Aug 06 2019   P-value (F-stat)                0.0000
Time:                        14:32:22   Distribution:                  chi2(7)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          4.7120     0.1982     23.778     0.0000      4.3236      5.1005
exper          0.0239     0.0040     5.9135     0.00

In [10]:
from linearmodels.iv.results import compare
print(compare({'OLS':OLS,'Proxy': Proxy,
   'IV_IQ': IV_IQ, 'IV_IQ_KWW': IV_IQ_KWW}))

                              Model Comparison                              
                             IV_IQ     IV_IQ_KWW           OLS         Proxy
----------------------------------------------------------------------------
Dep. Variable                lwage         lwage         lwage         lwage
Estimator                   IV-GMM        IV-GMM           OLS           OLS
No. Observations               935           935           935           935
Cov. Est.                   robust        robust        robust        robust
R-squared                   0.2106        0.2152        0.2526        0.2628
Adj. R-squared              0.2047        0.2092        0.2469        0.2564
F-statistic                 285.63        310.47        358.89        391.81
P-value (F-stat)            0.0000        0.0000        0.0000        0.0000
==================     ===========   ===========   ===========   ===========
const                       4.6730        4.7120        5.3955        5.1764

In [11]:
KWWasIVforIQ = IVGMM(df.lwage, df[Xs+['educ']], 
                     df.iq, df['kww']).fit()
print(KWWasIVforIQ)

                          IV-GMM Estimation Summary                           
Dep. Variable:                  lwage   R-squared:                      0.1900
Estimator:                     IV-GMM   Adj. R-squared:                 0.1830
No. Observations:                 935   F-statistic:                    356.33
Date:                Tue, Aug 06 2019   P-value (F-stat)                0.0000
Time:                        14:32:22   Distribution:                  chi2(8)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          4.5925     0.3501     13.117     0.0000      3.9063      5.2786
exper          0.0144     0.0034     4.2234     0.00

In [12]:
IQasIVforKWW = IVGMM(df.lwage, df[Xs+['educ']],
                     df.kww, df['iq']).fit()
print(IQasIVforKWW)

                          IV-GMM Estimation Summary                           
Dep. Variable:                  lwage   R-squared:                      0.0968
Estimator:                     IV-GMM   Adj. R-squared:                 0.0890
No. Observations:                 935   F-statistic:                    298.50
Date:                Tue, Aug 06 2019   P-value (F-stat)                0.0000
Time:                        14:32:22   Distribution:                  chi2(8)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          5.1700     0.1357     38.095     0.0000      4.9040      5.4360
exper          0.0029     0.0049     0.5892     0.55